## **_Workspace_**

In [174]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup as bs
import yfinance as yf
import numpy as np
import pickle

### **_DataLoader_**

In [175]:
# DataLoader class
class DataLoader():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Live data scraper
    def Live(self, symbol, choice):
        
		# Checking if price is chosen
        if choice == "price":
            
			# Setting variables to access the data
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{symbol}"
            
			# Extracting and parsing the HTML code
            request = requests.get(url, headers=headers)
            htmlcode = bs(request.text, "html.parser")
            
			# Finding and returning the price
            return float(htmlcode.find("fin-streamer", {"class": "livePrice svelte-mgkamr"}).text.replace(",", ''))

		# Checking if market capitalisation is chosen
        elif choice == "mcap":

			# Setting variables to access the data
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{symbol}"

			# Extracting and parsing the HTML code
            request = requests.get(url, headers=headers)
            htmlcode = bs(request.text, "html.parser")

            # Extracting the market capitalisation expression
            cap = (htmlcode.find("fin-streamer", {"data-field": "marketCap"}).text.replace(" ", ''))    
            
			# Creating a dictionary of suffix multipliers
            multipliers = {
            	'K': 1000,
            	'M': 1000000,
        		'B': 1000000000,
        		'T': 1000000000000
            }
            
			# Checking if there is suffix on the extracted expression
            if cap[-1] in multipliers:
                
				# Extracting and using the multiplier
                multiplier = multipliers[cap[-1]]
                value = float(cap[:-1]) * multiplier
                return int(value)
            
			# Otherwise there's no suffix so returning the raw extraction
            else:
                
                # Converting the cap to a float
                return float(cap)
            
    # Past data scraper
    def Past(self, universe):

		# Download and extract the close data
        data = yf.download(universe, start=datetime.today()-timedelta(30), end=datetime.today())
        closes = data["Close"]
        
		# Returning the close prices
        return closes

### **_Stock Screener_**

In [176]:
# Stock screener class
class Screener():
    
	# Initialiser
    def __init__(self):
        pass
        
	# Method to actually screen the stocks
    def Screen(self, data, count):
        
		# Getting the bounds
        before=data.loc[datetime.today()-timedelta(30):datetime.today()].iloc[0]
        after=data.loc[datetime.today()-timedelta(30):datetime.today()].iloc[-1]
        
		# Calculating the monthly returns
        returns = (((after-before) / before) * 100).sort_values(ascending=False)
        
		# Filtering out the best performing
        return returns.iloc[:count]

### **_Allocator_**

In [177]:
# Allocator class
class Allocator():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Method to allocate capital to stcoks according to Mcap weighting
    def Allocate(self, screened, capital, loader):
        
        # Creating lists for storage of prices and caps
        prices = []
        caps = []

		# Looping over the keys to extract prices for each
        for index in range(len(screened)):
            
			# Updating the lists
            prices.append(loader.Live(screened.keys()[index], "price"))
            caps.append(loader.Live(screened.keys()[index], "mcap"))

        # Converting the lists to arrays
        prices = np.array(prices)
        caps = np.array(caps)

		# Calculations for the respective quantities
        totalcap = sum(caps)
        proportions = caps / totalcap
        allocations = proportions * capital
        quantities = allocations // prices

		# Sorting everything into a dataframe
        details = pd.DataFrame({"Symbols":screened.keys(),
                                "Market Cap":caps,
                                "Proportions":proportions,
                                "Allocations":allocations,
                                "Quantities":quantities,
                                "Prices":prices})
        
		# Returning this dataframe to the user
        return details

### **_OrderManager_**

In [178]:
# Order manager class
class OrderManager():
    
	# Initialiser
    def __init__(self, loader):
        
		# Setting the dataloader
        self.loader = loader
    
	# Method to create an order
    def MakeOrder(self, symbol, quantity, side):
        
		# Creating a data loader and extracting the live price
        price = self.loader.Live(symbol, "price")
        
		# Getting today's date and time
        date = datetime.today()
        
		# Creating an order bracket
        bracket = pd.DataFrame({"Date":[date],
                                "Symbol":[symbol],
                                "Price":[price],
                                "Quantity":[quantity],
                                "Action":[side]})
        
		# Returning an order dataframe for the user
        return bracket

### **_Current Gains_**

In [179]:
# Calculating realised, unrealised, long and short term gains or losses
class Calculator():
    
	# Initialiser
    def __init__(self):
        pass
    
	# Method for calculating the gains or losses
    def Calculate(self, trades, current):
        
		# Calculating realised and unrealised, short and long term gains
        def CalculateGains(trades, current_price):
    
            # Initialise different gains storage
            realised_long_term_gains = 0
            realised_short_term_gains = 0
            unrealised_long_term_gains = 0
            unrealised_short_term_gains = 0

            # These lists contain their respective long positions
            all_long_positions = []
            short_term_long_positions = []
            long_term_long_positions = []

            # Looping over all trades
            for i in range(len(trades)):
    
                # Extracting a row for easier indexing
                row = trades.iloc[i]
    
                # Checking if it's a buy
                if row["Action"] == "buy":
    
                    # Appending all buys no matter the term
                    all_long_positions.append(row.copy())
    
                # Checking if it's a sell
                elif row["Action"] == "sell":
    
                    # Gathering specific sell details
                    og_shares_sold = row["Quantity"]
                    shares_sold = row["Quantity"]
                    price_sold = row["Price"]
                    date_sold = row["Date"]
                    total_short_term_cost_basis = 0
                    total_long_term_cost_basis = 0

                    # Reset positions lists
                    short_term_long_positions = []
                    long_term_long_positions = []

                    # Checking for long term or short term buys
                    for pos in all_long_positions:
    
                        # If the purchase was made more than a year before the sale
                        if pos["Date"] < (date_sold - timedelta(365)):
    
                            # Adding to the long term list
                            long_term_long_positions.append(pos)

                        # Otherwise                
                        else:
    
                            # Adding to the short term list
                            short_term_long_positions.append(pos)

                    # Process the shares sold for long term
                    while shares_sold > 0 and long_term_long_positions:
    
                        # Extracting the oldest long term buy
                        oldest_long_term_buy = long_term_long_positions[0]
    
                        # Checking if there are more shares sold than the quantity of the oldest buy
                        if shares_sold >= oldest_long_term_buy["Quantity"]:
    
                            # Calculating the cost basis
                            total_long_term_cost_basis += (oldest_long_term_buy["Quantity"] * oldest_long_term_buy["Price"])
    
                            # Updating the amount of shares left to handle
                            shares_sold -= oldest_long_term_buy["Quantity"]
    
                            # All oldest shares were used so they can be discarded
                            all_long_positions.remove(oldest_long_term_buy)
                            long_term_long_positions.pop(0)
    
                        # Otherwise deal with remaining shares
                        else:
    
                            # Calculating the basis as the remaining shares to sell times oldest price
                            total_long_term_cost_basis += (shares_sold * oldest_long_term_buy["Price"])
    
                            # Updating the amount of shares left on the oldest buy
                            oldest_long_term_buy["Quantity"] -= shares_sold

                            # No shares left to sell so setting it to zero which exits the loop
                            shares_sold = 0

                    # Total long term proceeds - the accurate cost basis
                    long_term_sold = og_shares_sold - shares_sold
                    long_term_proceeds = long_term_sold * price_sold
                    realised_long_term_gains += long_term_proceeds - total_long_term_cost_basis
    
                    # Reset shares sold for short-term calculation
                    shares_sold = og_shares_sold - long_term_sold

                    # Process the shares sold for short term
                    while shares_sold > 0 and short_term_long_positions:
    
                        # Extracting the oldest short term buy
                        oldest_short_term_buy = short_term_long_positions[0]
    
                        # Checking if there are more shares sold than the quantity of the oldest buy
                        if shares_sold >= oldest_short_term_buy["Quantity"]:
    
                            # Calculating the cost basis
                            total_short_term_cost_basis += (oldest_short_term_buy["Quantity"] * oldest_short_term_buy["Price"])
    
                            # Updating the amount of shares left to handle
                            shares_sold -= oldest_short_term_buy["Quantity"]
    
                            # All oldest shares were used so they can be discarded
                            all_long_positions.remove(oldest_short_term_buy)
                            short_term_long_positions.pop(0)
    
                        # Otherwise deal with remaining shares
                        else:
    
                            # Calculating the basis as the remaining shares to sell times oldest price
                            total_short_term_cost_basis += (shares_sold * oldest_short_term_buy["Price"])
    
                            # Updating the amount of shares left on the oldest buy
                            oldest_short_term_buy["Quantity"] -= shares_sold

                            # No shares left to sell so setting it to zero which exits the loop
                            shares_sold = 0

                    # Total short term proceeds - the accurate cost basis
                    short_term_sold = og_shares_sold - long_term_sold
                    short_term_proceeds = short_term_sold * price_sold
                    realised_short_term_gains += short_term_proceeds - total_short_term_cost_basis
            
            # Looping over the
            for position in all_long_positions:
        
		        # Calculating the current value of these positions
                current_value = position["Quantity"] * current_price
        
		        # Checking if the position is long or short term
                if position["Date"] < (datetime.today()-timedelta(365)):
            
			        # Updating the unrealised long term gains
                    unrealised_long_term_gains += current_value - (position["Quantity"] * position["Price"])
            
		        # Otherwise it's short term
                else:
            
			        # Updating the unrealised short term gains
                    unrealised_short_term_gains += current_value - (position["Quantity"] * position["Price"])
            
            # Returning the all the individual gains    
            return round(realised_long_term_gains, 2), round(realised_short_term_gains, 2), round(unrealised_long_term_gains, 2), round(unrealised_short_term_gains, 2)
		
		# Extracting the tuple of all gains
        all_gains = CalculateGains(trades, current)
        
		# Returning the gains to the user
        return all_gains       

### **_Wash Sale_**

In [180]:
# Checking trades for wash sale
class Checker():
    
	# Initialiser
    def __init__(Self):
        pass
    
	# Method for checking
    def Check(self, trades):
        
		# Setting time variables
        today = datetime.today()
        delta = timedelta(30)
        
		# Looping over the trades
        for i in range(len(trades)):
            
			# Isolating the trade date
            date = trades.iloc[i]["Date"]
            
            # Setting the status
            safe = True

			# Check if it falls within the period
            if (today-delta) <= date <= (today+delta):
                
				# Updating the status and exiting the loop
                safe = False
                break
                
		# Checking if the final status is safe
        if safe:
            
			# Returning true if they don't violate washsale
            return True
        
		# Otherwise...
        else:
            
			# Returning false as they don't violate washsale
            return False

### **_Calculate Tax_**

In [181]:
# Calculating the tax due before and after a harvest
class Taxer():
    
    # Initialiser
    def __init__(self):
        pass
    
    # Method for calculating
    def Tax(self, gains, carry_forward, print_info):
        
        # Extracting the specific gains and setting some initial values
        long_term_realised, short_term_realised, long_term_unrealised, short_term_unrealised = gains
        long_term_remaining = long_term_realised
        short_term_remaining = short_term_realised
        final_forward = carry_forward
        og = carry_forward
        
		# Checking if the user wants printouts
        if print_info == True:
            
		    # Informing user of their inputs
            print(f"LT Realised Gains:   {long_term_realised}")
            print(f"ST Realised Gains:   {short_term_realised}")
            print(f"LT Unrealised Gains: {long_term_unrealised}")
            print(f"ST Unrealised Gains: {short_term_unrealised}")
        
        # Calculate initial tax before considering unrealized losses
        long_term_tax_before = max(0, long_term_realised) * 0.1
        short_term_tax_before = max(0, short_term_realised) * 0.2
        
		# Checking if the user wants printouts
        if print_info == True:
            
		    # Informing user of the tax due before a harvest
            print(f"LT before: {long_term_tax_before}")
            print(f"ST before: {short_term_tax_before}")
        
		# Handling long term unrealized losses
        if long_term_unrealised < 0 and long_term_realised > 0:
            
			# If the losses don't cancel the gains and more
            if long_term_realised >= abs(long_term_unrealised):
                
				# Calculating the tax and other important variables
                long_term_tax_after = (long_term_realised + long_term_unrealised) * 0.1
                long_term_remaining_loss = 0
                long_term_remaining = (long_term_realised + long_term_unrealised)
                # print("1. LT if")
                
			# Otherwise the losses completely cancel the gains and can be used elsewhere
            else:
                
				# Setting tax after to 0 and finding other important variables
                long_term_tax_after = 0
                long_term_remaining_loss = long_term_realised + long_term_unrealised
                long_term_remaining = 0
                # print("1. LT else")
                
		# No unrealised losses
        else:
            
			# Checking if gains are positive
            if long_term_unrealised + long_term_realised >= 0:
                
				# Calculating the tax after and other important variables
                long_term_tax_after = (long_term_realised + long_term_unrealised) * 0.1
                long_term_remaining_loss = 0
                long_term_remaining = (long_term_realised + long_term_unrealised)
                # print("2. LT if")
                
			# Otherwise there are leftover losses
            else:
                
				# Setting tax after to 0 and finding other important variables
                long_term_tax_after = 0
                long_term_remaining_loss = (long_term_realised + long_term_unrealised)
                long_term_remaining = 0
                # print("2. LT else")
                
        # Handling short term unrealised losses
        if short_term_unrealised < 0 and short_term_realised > 0:
            
			# If the losses don't cancel the gains and more
            if short_term_realised >= abs(short_term_unrealised):
                
				# Calculating the tax and other important variables
                short_term_tax_after = (short_term_realised + short_term_unrealised) * 0.2
                short_term_remaining_loss = 0
                short_term_remaining = (short_term_realised + short_term_unrealised)
                # print("3. ST if")
                
			# Otherwise the losses completely cancel out the gains and can be used elsewhere
            else:
                
				# Setting tax after to 0 and finding other important variables
                short_term_tax_after = 0
                short_term_remaining_loss = short_term_realised + short_term_unrealised
                short_term_remaining = 0
                # print("3. ST else")
                
		# No unrealised losses
        else:
            
            # Checking if gains are positive
            if short_term_unrealised + short_term_realised >= 0:
                
				# Calculating the tax after and other important variables
                short_term_tax_after = (short_term_realised + short_term_unrealised) * 0.2
                short_term_remaining_loss = 0
                short_term_remaining = (short_term_realised + short_term_unrealised)
                # print("4. ST if")
                
			# Otherwise there are leftover losses
            else:
                
				# Setting tax after to 0 and finding other important variables
                short_term_tax_after = 0
                short_term_remaining_loss = (short_term_realised + short_term_unrealised)
                short_term_remaining = 0
                # print("4. ST else")
            
		# Handling any remaining long term losses
        if long_term_remaining_loss < 0:
            
            # If there are still short term gains to offset
            if short_term_remaining > 0:

				# If the losses don't cancel the gains and more
                if short_term_remaining >= abs(long_term_remaining_loss):
                    
					# Calculating the new tax and finding other important variables
                    short_term_tax_after = (short_term_remaining + long_term_remaining_loss) * 0.2
                    short_term_remaining = (short_term_remaining + long_term_remaining_loss)
                    long_term_remaining_loss = 0
                    # print("5. if")
                
				# Otherwise there are still leftover losses
                else:
                    
					# Setting the remaining tax to 0 and setting other variables
                    short_term_tax_after = 0
                    long_term_remaining_loss = short_term_remaining + long_term_remaining_loss
                    short_term_remaining = 0
                    # print("5. else")

			# There are no more gains to offset
            else:
                # print("5. no gains")
                pass
                
		# Handling any remaining short term losses
        if short_term_remaining_loss < 0:
            
			# If there are still long term gains to offset
            if long_term_remaining > 0:
                
				# If the losses don't cancel the gains and more
                if long_term_remaining >= abs(short_term_remaining_loss):
                                        
					# Calculating the new tax and finding other important variables
                    long_term_tax_after = (long_term_remaining + short_term_remaining_loss) * 0.1
                    long_term_remaining = (long_term_remaining + short_term_remaining_loss)
                    short_term_remaining_loss = 0
                    # print("6. if")
                    
				# Otherwise there are still leftover losses
                else:
                    
					# Setting the remaining tax to 0 and setting other variables
                    long_term_tax_after = 0
                    short_term_remaining_loss = long_term_remaining + short_term_remaining_loss
                    long_term_remaining = 0
                    # print("6. else")
                    
			# There are no more gains to offset
            else:
                # print("6. no gains")
                pass
        
		# Checking if there are any carry forward losses
        if carry_forward < 0:
            
			# Checking if any short term gains left (offset them first because of the higher rate)
            if short_term_remaining > 0:
                
                # If the losses don't cancel the gains and more
                if short_term_remaining >= abs(carry_forward):
                    
                    # Calculating the new tax and setting other important variables
                    short_term_tax_after = (short_term_remaining + carry_forward) * 0.2
                    short_term_remaining = (short_term_remaining + carry_forward)
                    carry_forward = 0
                    # print("7. ST CF if")
                
				# Otherwise there are still leftover losses
                else:
                    
					# Setting the remaining tax to 0 and setting other variables
                    short_term_tax_after = 0
                    carry_forward = short_term_remaining + carry_forward
                    short_term_remaining = 0
                    # print("7. ST CF else")

			# There are no more gains to offset
            else:
                # print("7. ST CF no gains")
                pass

        # If there are still more carry forward
        if carry_forward < 0:
            
			# Checking if there are any long term gains to offset
            if long_term_remaining > 0:
                
				# If the losses don't cancel the gains and more
                if long_term_remaining >= abs(carry_forward):
                    
                    # Calculating the new tax and setting other important variables
                    long_term_tax_after = (long_term_remaining + carry_forward) * 0.1
                    long_term_remaining	= (long_term_remaining + carry_forward)
                    carry_forward = 0
                    # print("8. LT CF if")
                
				# Otherwise there are still leftover losses
                else:
                    
					# Setting the remaining tax to 0 ans setting other variables
                    long_term_tax_after = 0
                    carry_forward = long_term_remaining + carry_forward
                    long_term_remaining = 0
                    # print("8. LT CF else")
                    
			# There are no more gains to offset
            else:
                # print("8. LT CF no gains")
                pass

        # Calculating the amount of losses left to carry forward 
        finalcarry = sum(gains)
        finalcarry = finalcarry + final_forward
        
		# Calculating the savings used to determine worth of harvest
        long_term_savings = long_term_tax_before - long_term_tax_after
        short_term_savings = short_term_tax_before - short_term_tax_after
        total_tax_before = long_term_tax_before + short_term_tax_before
        total_tax_after = long_term_tax_after + short_term_tax_after
        total_tax_savings = total_tax_before - total_tax_after
        try:
            total_tax_percentage_savings = (round(total_tax_savings / total_tax_before, 2)) * 100
        except ZeroDivisionError as e:
            total_tax_percentage_savings = 0
            print(e)
            pass
        
		# Calculating the gains
        total_gains_before = long_term_realised + short_term_realised
        total_gains_after = total_gains_before + long_term_unrealised + short_term_unrealised + min(og, 0)
        net_gains_before = total_gains_before - (total_tax_before)
        net_gains_after = total_gains_after - (total_tax_after)
        
        # Checking if the user wants printouts
        if print_info == True:
            
		    # Printing final after taxes and any carry forward losses
            print(f"LT after:            {long_term_tax_after}")
            print(f"ST after:            {short_term_tax_after}")
            print(f"LT saved:            {long_term_savings}")
            print(f"ST saved:            {short_term_savings}")
            print(f"Total savings:       {total_tax_savings}")
            print(f"Total savings:       %{total_tax_percentage_savings}")
            print(f"Gross gains before:  {total_gains_before}")
            print(f"Gross gains after:   {total_gains_after}")
            print(f"Net gains before:    {net_gains_before}")
            print(f"Net gains after:     {net_gains_after}")
            print(f"Carry forward:       {min(finalcarry, 0)}")
            
		# Returning the saving percentage, list of taxes and carry forward losses
        return total_tax_percentage_savings, [long_term_tax_before, short_term_tax_before, long_term_tax_after, short_term_tax_after], min(finalcarry, 0)

### **_Engine_**

In [182]:
# Tax loss harvesting engine
class TaxLossHarvester():
    
	# Initialiser
    def __init__(self, checker, calculator, taxer, loader):
        
		# Setting the base functions for harvesting
        self.calculator = calculator
        self.checker = checker
        self.loader = loader
        self.taxer = taxer
    
	# Method for harvesting
    def Harvest(self, trades, threshold, carry, info):
        
		# Checking if the trades violate the wash sale
        status = self.checker.Check(trades)
        
		# Checking if the status is good
        if status == True:

            # Extracting the symbol and its price
            symbol = trades.iloc[0]["Symbol"]
            current = self.loader.Live(symbol, "price")

            # Extracting the realised and unrealised gains
            gains = self.calculator.Calculate(trades, current)
            
            # Calculating the taxes due
            savings, taxes, carry = self.taxer.Tax(gains, carry, info)

			# Checking conditions for the harvest
            if savings >= threshold:
                
				# It's good for harvesting
                return True, carry

			# Otherwise no harvest
            else:

                # Not good for harvesting
                return False, carry
            
		# Otherwise no harvest
        else:
            
			# Not good for harvesting
            return False, carry

### **_Rebalancer_**

In [183]:
# Class for rebalancing
class Rebalancer():
    
	# Initialiser
    def __init__(self, universe, capital, size, loader, screener, allocator):
        
		# Initialising the tools
        self.loader = loader
        self.screener = screener
        self.allocator = allocator
        
		# Setting portfolio attributes
        self.universe = universe.copy()
        self.capital = capital
        self.size = size
    
	# Method for initialising the portfolio's stocks
    def RebalanceInit(self):

		# Calculating the quantities for given stocks
        stocks = self.loader.Past(self.universe)
        screened = self.screener.Screen(stocks, self.size)
        allocated = self.allocator.Allocate(screened, self.capital, self.loader)
        specifics = allocated[["Symbols", "Quantities"]].copy()
        print(allocated)
        
		# Returning the allocated dataframe which contains quantities
        return specifics

	# Method for rebalancing based on TLH engine
    def RebalanceTLH(self, harvested, assets):
        
		# Setting the available universe to everything bar the harvested
        available = [asset for asset in self.universe if asset not in harvested]
       
	    # Calculating the quantities for available
        stocks = self.loader.Past(available)
        screened = self.screener.Screen(stocks, self.size)
        allocated = self.allocator.Allocate(screened, self.capital, self.loader)
        specifics = allocated[["Symbols", "Quantities"]].copy()
        print(allocated)
        
	    # Finding the stocks in assets that aren't in allocated
        for i in range(len(assets)):
            
            # Passing the filler value
            if assets.iloc[i]["Symbol"] != "filler":

		 	    # Extracting the specifics of assets
                symbol = assets.iloc[i]["Symbol"]
                quantity = assets.iloc[i]["Quantity"]

		  	    # Checking if it's not in allocated
                if symbol not in specifics["Symbols"].values:
                
		  	    	# Updating the lists
                    specifics.loc[len(specifics)+1] = [symbol, -quantity]
                    
			# Otherwise pass
            else:
                pass
        
		# Looping over the allocated
        for i in range(len(allocated)):
            
		 	# Extracting specifics
            symbol = allocated.iloc[i]["Symbols"]
            quantity = allocated.iloc[i]["Quantities"]
            
		 	# Checking if this asset is in the portfolio
            if symbol in assets["Symbol"].values:
                
                # Calculating the difference between the quantities
                difference = quantity - assets.loc[assets["Symbol"] == symbol]["Quantity"]
                specifics.loc[specifics["Symbols"]==symbol, "Quantities"] = difference.iloc[0]
        
        # Returning the allocated dataframe which contains quantities
        return specifics
    
	# Method for a regular rebalance
    def RebalanceReg(self, assets):
        
		# Calculating the quantities for given stocks
        stocks = self.loader.Past(self.universe)
        screened = self.screener.Screen(stocks, self.size)
        allocated = self.allocator.Allocate(screened, self.capital, self.loader)
        specifics = allocated[["Symbols", "Quantities"]].copy()
        print(allocated)
            
		# Finding the stocks in assets that aren't in allocated
        for i in range(len(assets)):
            
            # Passing the filler value
            if assets.iloc[i]["Symbol"] != "filler":

			    # Extracting the specifics of assets
                symbol = assets.iloc[i]["Symbol"]
                quantity = assets.iloc[i]["Quantity"]

		 	    # Checking if it's not in allocated
                if symbol not in specifics["Symbols"].values:
                
		 		    # Updating the lists
                    specifics.loc[len(specifics)+1] = [symbol, -quantity]
                    
			# Otherwise pass
            else:
                pass
        
		# Finding the stocks in allocated that are in assets aready
        for i in range(len(allocated)):
            
		  	# Extracting the specifics of allocated
            symbol = allocated.iloc[i]["Symbols"]
            quantity = allocated.iloc[i]["Quantities"]
            
            # Checking if it's in assets
            if symbol in assets["Symbol"].values:
               
                # Calculating the difference in quantity between the two
                difference = quantity - assets.loc[assets["Symbol"] == symbol]["Quantity"]
                specifics.loc[specifics["Symbols"]==symbol, "Quantities"] = difference.iloc[0]
                
        # Returning the quantities to be traded
        return specifics

### **_Portfolio_**

In [184]:
# Portfolio class
class Portfolio():
    
	# Initialiser, adds stocks
    def __init__(self, universe, capital, size, file=None):
        
        # If no file is provided, run regular initialisation protocol
        if file == None:

		    # Setting portfolio attributes
            self.carry = 0
            self.size = size
            self.capital = capital
            self.starting_capital = capital
            self.universe = universe.copy()
            self.available_universe = universe.copy()
        
		    # Setting manager dataframes for tracking trades and assets
            self.trades = pd.DataFrame({"Date":["filler"], "Symbol":["filler"], "Price":["filler"], "Quantity":["filler"], "Action":["filler"]})
            self.assets = pd.DataFrame({"Symbol":["filler"], "Quantity":["filler"]})

		    # Setting the tools of the portfolio
            self.taxer = Taxer()
            self.checker = Checker()
            self.calculator = Calculator()
            self.loader = DataLoader()
            self.screener = Screener()
            self.allocator = Allocator()
            self.manager = OrderManager(self.loader)
            self.engine = TaxLossHarvester(self.checker, self.calculator, self.taxer, self.loader)
            self.rebalancer = Rebalancer(self.universe, self.capital, self.size, self.loader, self.screener, self.allocator)

		    # Getting the quantities of stocks to initialise the portfolio
            allocated = self.rebalancer.RebalanceInit()
        
		    # Looping over the chosen stocks and adding them to the portfolio
            for index in range(len(allocated)):
            
			    # Adding the asset
                self.BuyAsset(allocated["Symbols"].iloc[index], allocated["Quantities"].iloc[index])
                
		# Otherwise attempt to load provided file data
        else:
            
			# Retrieving function
            self.LoadPortfolio(file)
            
			# Setting the tools of the portfolio
            self.taxer = Taxer()
            self.checker = Checker()
            self.calculator = Calculator()
            self.loader = DataLoader()
            self.screener = Screener()
            self.allocator = Allocator()
            self.manager = OrderManager(self.loader)
            self.engine = TaxLossHarvester(self.checker, self.calculator, self.taxer, self.loader)
            self.rebalancer = Rebalancer(self.universe, self.capital, self.size, self.loader, self.screener, self.allocator)
        
	# Method to buy an asset for the portfolio
    def BuyAsset(self, symbol, quantity):
        
		# Creating the order
        order = self.manager.MakeOrder(symbol, quantity, "buy")
        
		# If the asset isn't already present on the portfolio
        if order["Symbol"][0] not in self.assets["Symbol"].values:
            
			# Checking if a new stock breaches the portfolio size
            if (len(self.assets)-1) >= self.size:
                print("Portfolio size limit reached. Cannot add new asset")
                return
            
			# Adding the asset to the portfolio asset list
            update = pd.DataFrame({"Symbol":[order["Symbol"][0]], "Quantity":[order["Quantity"].values[0]]})
            self.assets = pd.concat([self.assets, update]).reset_index(drop=True)
            
			# Logging the trade
            self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
            
			# Updating the list of available stocks that aren't under the portfolio
            self.available_universe.remove(order["Symbol"][0])
        
		# Otherwise if the asset is already in the portfolio
        else:
            
			# Updating the quantity of the assets
            current_quantity = self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"].values[0]
            new_quantity = current_quantity + order["Quantity"].values[0]
            self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"] = new_quantity
            
			# Logging the other trades
            self.trades = pd.concat([self.trades, order]).reset_index(drop=True)

		# Updating the capital
        self.capital -= float((order["Quantity"] * order["Price"]).iloc[0])
    
	# Method to sell an asset for the portfolio
    def SellAsset(self, symbol, quantity):
        
        # Creating the order
        order = self.manager.MakeOrder(symbol, quantity, "sell")

		# If the asset is already present on the portfolio
        if order["Symbol"][0] in self.assets["Symbol"].values:
            
			# Calculating the new portfolio quantity of the asset
            current_quantity = self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"].values[0]
            new_quantity = current_quantity - order["Quantity"].values[0]
            
			# Checking if the sale quantity was greater than current quantity (Short sale)
            if new_quantity < 0:
                
				# Inform the user and exit
                print("Quantity too great, no short sales allowed")
            
			# If the sale cancels out position completely then remove it from assets
            elif new_quantity == 0:
                
				# Removing the asset from the portfolio
                self.assets = self.assets.loc[self.assets["Symbol"] != order["Symbol"][0]].reset_index(drop=True)
                
				# Logging the trade
                self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
                
				# Updating the capital
                self.capital += (order["Quantity"] * order["Price"])
                
				# Updating the stocks that we don't have under the portfolio
                self.available_universe.append(order["Symbol"][0])
            
			# Otherwise if there would still be a remaining amount
            else:
                
				# Updating the asset's quantity on the portfolio
                self.assets.loc[self.assets["Symbol"]==order["Symbol"][0], "Quantity"] = new_quantity
                
				# Combining with the other trades
                self.trades = pd.concat([self.trades, order]).reset_index(drop=True)
                
				# Updating the capital
                self.capital += float((order["Quantity"] * order["Price"]).iloc[0])
            
		# Otherwise the position isn't present and shorting isn't allowed so...
        else:
            
			# Informing the user
            print("This position is not present and shorting isn't available")
    
	# Method to run the TLH engine
    def Harvester(self, threshold):
		
        # Setting a copy of the assets
        temp_assets = self.assets.copy()
        harvests = []

		# Looping over all assets to assess quality of harvest
        for i in range(len(temp_assets)):
            
			# Extracting the individual asset and information
            current_asset = self.assets.iloc[i]["Symbol"]
            current_quantity = self.assets.iloc[i]["Quantity"]
            current_trades = self.trades.loc[self.trades["Symbol"] == current_asset]
            
            # Passing the filler value
            if current_asset != "filler":

			    # Checking if a harvest is allowed and good
                status, self.carry = self.engine.Harvest(current_trades, threshold, self.carry, True)
                print()
                print(f"Asset: {current_asset}, Harvest: {status}")
                if status == True:
					
				    # Harvesting
                    self.SellAsset(current_asset, current_quantity)
                    harvests.append(current_asset)
                
        # Getting the quantities to trade for the rebalance
        allocated = self.rebalancer.RebalanceTLH(harvests, temp_assets)
        
		# Looping over the allocated and checking if we need to buy or sell
        for index in range(len(allocated)):
            
			# Checking for a buy
            if allocated["Quantities"].iloc[index] > 0:
                
				# Buying the given quantity of given asset
                self.BuyAsset(allocated["Symbols"].iloc[index], allocated["Quantities"].iloc[index])
                
			# Checking for a sell
            elif allocated["Quantities"].iloc[index] < 0:
                
				# Selling the given quantity of given asset
                self.SellAsset(allocated["Symbols"].iloc[index], abs(allocated["Quantities"].iloc[index]))
                
			# Otherwise the quantity is 0
            else:
                pass
            
	# Method to rebalance the portfolio
    def Rebalance(self):
        
		# Getting the quantities to trade for the rebalance
        allocated = self.rebalancer.RebalanceReg(self.assets)
        
		# Looping over the allocated and checking if we need to buy or sell
        for index in range(len(allocated)):
            
			# Checking for a buy
            if allocated["Quantities"].iloc[index] > 0:
                
				# Buying the given quantity of given asset
                self.BuyAsset(allocated["Symbols"].iloc[index], allocated["Quantities"].iloc[index])
                
			# Checking for a sell
            elif allocated["Quantities"].iloc[index] < 0:
                
				# Selling the given quantity of given asset
                self.SellAsset(allocated["Symbols"].iloc[index], abs(allocated["Quantities"].iloc[index]))
                
			# Otherwise the quantity is 0
            else:
                pass
            
	# Method to save current portfolio data
    def SavePortfolio(self, filename):
        
        # Extracting the attributes
        data = {"carry":self.carry,
                "size":self.size,
                "capital":self.capital,
                "starting_capital":self.starting_capital,
                "universe":self.universe,
                "available_universe":self.available_universe,
                "trades":self.trades,
                "assets":self.assets}

		# Context manager
        with open(filename, "wb") as f:
            pickle.dump(data, f)
            
	# Method for loading previous portfolio data
    def LoadPortfolio(self, filename):
        
		# Attempting to retrieve data
        try:
            
			# Context manager
            with open(filename, "rb") as f:
                
				# Loading the data and pasting it to the portfolio
                data = pickle.load(f)
                self.carry = data["carry"]
                self.size = data["size"]
                self.capital = data["capital"]
                self.starting_capital = data["starting_capital"]
                self.universe = data["universe"]
                self.available_universe = data["available_universe"]
                self.trades = data["trades"]
                self.assets = data["assets"]
        
		# Handling a file not found error
        except FileNotFoundError:
            print("Save file not found.")

In [185]:
# Initialising the portfolio
MyPf = Portfolio(["AAPL", "AMZN", "MSFT", "META", "TSLA", "GOOG", "NVDA", "NFLX"], 10000, 3)

[*********************100%%**********************]  8 of 8 completed


  Symbols     Market Cap  Proportions  Allocations  Quantities  Prices
0    TSLA   792516000000     0.121022  1210.222285         4.0  248.50
1    AAPL  3510000000000     0.535999  5359.993012        23.0  228.88
2    GOOG  2246000000000     0.342978  3429.784702        18.0  182.62


In [186]:
# The current assets held within the portfolio
MyPf.assets

,Symbol,Quantity
0,filler,filler
1,TSLA,4.0
2,AAPL,23.0
3,GOOG,18.0


In [187]:
# All trades that have been made on this portfolio
MyPf.trades

,Date,Symbol,Price,Quantity,Action
0,filler,filler,filler,filler,filler
1,2024-07-18 14:06:10.022096,TSLA,248.5,4.0,buy
2,2024-07-18 14:06:11.035110,AAPL,228.88,23.0,buy
3,2024-07-18 14:06:11.947573,GOOG,182.62,18.0,buy


In [188]:
# Remaining capital
MyPf.capital

454.60000000000036

In [189]:
# Any extra losses that can be used for harvesting
MyPf.carry

0

In [190]:
# Add assets manually (Shouldn't do because certain hings like washsale aren't checked)
MyPf.BuyAsset("AAPL", 1)

In [191]:
# Use in-built gains calculator and data loader to find gains of a single asset
MyPf.calculator.Calculate(MyPf.trades[MyPf.trades["Symbol"]=="AAPL"], MyPf.loader.Live("AAPL", "price"))

(0, 0, 0, 0.0)

In [192]:
MyPf.assets

,Symbol,Quantity
0,filler,filler
1,TSLA,4.0
2,AAPL,24.0
3,GOOG,18.0


In [193]:
MyPf.trades

,Date,Symbol,Price,Quantity,Action
0,filler,filler,filler,filler,filler
1,2024-07-18 14:06:10.022096,TSLA,248.5,4.0,buy
2,2024-07-18 14:06:11.035110,AAPL,228.88,23.0,buy
3,2024-07-18 14:06:11.947573,GOOG,182.62,18.0,buy
4,2024-07-18 14:06:20.686785,AAPL,228.88,1,buy


In [165]:
# Saving portfolio's current state
MyPf.SavePortfolio("Example_PF.pkl")

In [154]:
# Harvesting any losses
MyPf.Harvester(0)

[*********************100%%**********************]  8 of 8 completed


Asset: TSLA, Harvest: False

Asset: AAPL, Harvest: False

Asset: GOOG, Harvest: False


<class 'int'>
  Symbols     Market Cap  Proportions  Allocations  Quantities  Prices
0    TSLA   792516000000     0.121022  1210.222285         4.0  248.50
1    AAPL  3510000000000     0.535999  5359.993012        23.0  228.88
2    GOOG  2246000000000     0.342978  3429.784702        18.0  182.62


In [155]:
MyPf.assets

,Symbol,Quantity
0,filler,filler
1,TSLA,4.0
2,AAPL,23.0
3,GOOG,18.0


In [144]:
# A regular rebalance of the portfolio
MyPf.Rebalance()

[*********************100%%**********************]  8 of 8 completed


<class 'int'>
  Symbols     Market Cap  Proportions  Allocations  Quantities  Prices
0    TSLA   792516000000     0.121022  1210.222285         4.0  248.50
1    AAPL  3510000000000     0.535999  5359.993012        23.0  228.88
2    GOOG  2246000000000     0.342978  3429.784702        18.0  182.62


In [145]:
MyPf.assets

,Symbol,Quantity
0,filler,filler
1,TSLA,4.0
2,AAPL,23.0
3,GOOG,18.0


In [194]:
# Loading a previous portfolio
OldPf = Portfolio(None, None, None, "Example_PF.pkl")

In [195]:
# Different trades compared to current portfolio
OldPf.trades

,Date,Symbol,Price,Quantity,Action
0,filler,filler,filler,filler,filler
1,2024-07-18 14:01:46.547887,TSLA,248.5,4.0,buy
2,2024-07-18 14:01:47.426894,AAPL,228.88,23.0,buy
3,2024-07-18 14:01:48.373974,GOOG,182.62,18.0,buy
4,2024-07-18 14:02:08.757294,AAPL,228.88,1,buy


In [196]:
# Different trades compared to old portfolio
MyPf.trades

,Date,Symbol,Price,Quantity,Action
0,filler,filler,filler,filler,filler
1,2024-07-18 14:06:10.022096,TSLA,248.5,4.0,buy
2,2024-07-18 14:06:11.035110,AAPL,228.88,23.0,buy
3,2024-07-18 14:06:11.947573,GOOG,182.62,18.0,buy
4,2024-07-18 14:06:20.686785,AAPL,228.88,1,buy
